In [141]:
import numpy as np
from numpy.linalg import inv, norm, svd, eig

In [142]:
def calc_a_phi_vect(M, phi):
    return np.exp(1j * np.arange(M)[:, np.newaxis] * phi)

In [143]:
def calcProjectionMatrix(A):
    # MATLAB version: A/(A'*A)*A';
    A_H = A.conj().T
    return A @ np.linalg.inv(A_H @ A) @ (A_H)

def calcOProjectionMatrix(A):
    """Compute the orthogonal projection matrix"""
    M = A.shape[0]
    return np.eye(M) - calcProjectionMatrix(A)

def calcCa(C):
    K = C.shape[1]
    Coproj = calcOProjectionMatrix(C)
    Ca = Coproj[:,K:]
    return Ca

In [144]:
N = 1000
M = 12

K = 2  # Number of desired wavefronts


desired_angles = np.pi * np.array([30, 70]) / 180.0  # Size is "K"
undesired_angles = np.pi * np.array([-20, 45]) / 180.0  # 2 interfering wavefronts

data = np.sign(np.random.randn(K, N))
interfering_data = np.sign(np.random.randn(2, N))

In [145]:
desired_steering_vec = calc_a_phi_vect(12,desired_angles)
undesired_steering_vec = calc_a_phi_vect(12,undesired_angles)

In [174]:
received_data = desired_steering_vec @ data + undesired_steering_vec @ interfering_data

C = desired_steering_vec
Ca = calcCa(C)
g = np.ones(K)

# Covariance matrix of the received Data
Ruu = received_data @ received_data.T.conj()
Rxx = Ca_H @ Ruu @ Ca

In [165]:
Ca_H = Ca.conj().T
C_H = C.conj().T

U = np.hstack([C, Ca])

v = inv(C_H @ C) @ g
wq = C @ v

#inv(C_H@C)@v

#(C_H @ Ca).round(2)
#(C @ v).round(2)
wopt = (1 - Ca @ inv(Ca_H @ Ruu @ Ca) @ Ca_H @ Ruu) @ C @ inv(C_H @ C) @ g
wopt2 = inv(Ruu) @ C @ inv(C_H @ inv(Ruu) @ C) @ g

#np.abs((C_H @ wopt))

q = inv(U) @ wopt
print(q[:2])
print(v)
#print(q[2:])

[-0.00203094-0.00235202j  0.01453807+0.00235202j]
[ 0.0731188-0.01182944j  0.0731188+0.01182944j]


In [152]:
wopt

array([ 0.92900805+0.55977992j,  0.39734199+0.3618499j ,
        0.64675496-0.97536141j, -0.06434083+0.08027274j,
       -0.47159180+0.34249413j,  0.54239964+0.95066064j,
        0.71080194-0.24796977j,  0.64541609+0.47539545j,
       -0.06441817-0.33409048j,  1.99344819-0.2914294j ,
       -1.55874332-0.58599089j,  0.90306113+1.34142446j])

In [153]:
wopt2

array([ 3.23589381+0.19184278j, -5.93077466-2.66245171j,
        4.53494067+2.99782854j, -2.21905443-1.42926953j,
        0.09824917+1.32244509j,  1.14594842-1.35834658j,
       -1.17742539-0.36821036j, -0.19090051+1.07213583j,
        0.03471576-0.28989741j,  0.18508210+0.02060899j,
        0.97164570-0.26677502j, -0.59852982-0.52179425j])

In [176]:
Px = Ca_H @ Ruu @ wq
wa = inv(Rxx) @ Px

In [180]:
dn = wq.conj().T @ received_data
error = dn - wa.conj().T @ Ca_H @ received_data
error.shape

(1000,)